In [140]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

In [141]:
raw_html = requests.get("http://www.tdcj.state.tx.us/death_row/dr_executed_offenders.html").content

In [142]:
soup_doc = BeautifulSoup(raw_html, "html.parser")  

In [143]:
tables = soup_doc.find(class_='tdcj_table indent').find_all('tr')

In [144]:
info = []
for table in tables[1:]:
    result = {}
    result['lastname'] = table.find_all('td')[3].text
    result['firstname'] = table.find_all('td')[4].text
    result['age'] = table.find_all('td')[6].text
    result['date'] = table.find_all('td')[7].text
    result['race'] = table.find_all('td')[8].text 
    url = table.find_all('td')[2].find('a')['href']
    result['url'] = table.find_all('td')[2].find('a')['href']
    

    
    info.append(result)
    
info
    

[{'lastname': 'Young',
  'firstname': 'Christopher',
  'age': '34',
  'date': '7/17/2018',
  'race': 'Black',
  'url': 'dr_info/youngchristopherlast.html'},
 {'lastname': 'Bible',
  'firstname': 'Danny',
  'age': '66',
  'date': '6/27/2018',
  'race': 'White',
  'url': 'dr_info/bibledannylast.html'},
 {'lastname': 'Castillo',
  'firstname': 'Juan',
  'age': '37',
  'date': '5/16/2018',
  'race': 'Hispanic',
  'url': 'dr_info/castillojuanlast.html'},
 {'lastname': 'Davila',
  'firstname': 'Erick',
  'age': '31',
  'date': '4/25/2018',
  'race': 'Black',
  'url': 'dr_info/davilaericklast.html'},
 {'lastname': 'Rodriguez, III',
  'firstname': 'Rosendo',
  'age': '38',
  'date': '3/27/2018',
  'race': 'Histpanic',
  'url': 'dr_info/rodriguezrosendolast.html'},
 {'lastname': 'Battaglia',
  'firstname': 'John',
  'age': '62',
  'date': '2/1/2018',
  'race': 'White',
  'url': 'dr_info/battagliajohnlast.html'},
 {'lastname': 'Rayford',
  'firstname': 'William',
  'age': '64',
  'date': '1/30/2

In [149]:
df1 = pd.read_csv('info.csv')

In [150]:
df1.head(10)

,age,date,firstname,lastname,race,url
0,34,7/17/2018,Christopher,Young,Black,dr_info/youngchristopherlast.html
1,66,6/27/2018,Danny,Bible,White,dr_info/bibledannylast.html
2,37,5/16/2018,Juan,Castillo,Hispanic,dr_info/castillojuanlast.html
3,31,4/25/2018,Erick,Davila,Black,dr_info/davilaericklast.html
4,38,3/27/2018,Rosendo,"Rodriguez, III",Histpanic,dr_info/rodriguezrosendolast.html
5,62,2/1/2018,John,Battaglia,White,dr_info/battagliajohnlast.html
6,64,1/30/2018,William,Rayford,Black,dr_info/rayfordwilliamlast.html
7,55,1/18/2018,Anthony,Shore,White,dr_info/shoreanthonylast.html
8,47,11/08/2017,Ruben,Cardenas,Hispanic,dr_info/cardenasrubenlast.html
9,38,10/12/2017,Robert,Pruett,White,dr_info/pruettrobertlast.html


In [151]:
html = requests.get('http://www.tdcj.state.tx.us/death_row/dr_info/pruettrobertlast.html').content
doc = BeautifulSoup(html, "html.parser")
results = doc.find('div', attrs={'id':'content_right'}).find_all('p')
for result in results:
    statement = result.text
    print(statement)

Date of Execution:
October 12, 2017
Offender:
Robert Lynn Pruett #999411
Last Statement:
I just want to let everyone in here to know I love you so much. I’ve hurt a lot of people and a lot of people have hurt me. I love y'all so much. Life don’t end here it goes on forever. I’ve had to learn lessons in life the hard way. One day there won’t be a need to hurt people. I love y'all so much. I’m ready to go but I’ll be back. Nighty night everybody, nighty night everybody. I’m done warden.


In [138]:
html = requests.get('http://www.tdcj.state.tx.us/death_row/dr_info/davilaericklast.html').content
doc = BeautifulSoup(html, "html.parser")
results = doc.find('div', attrs={'id':'content_right'}).find_all('p')

statement_list =[]
for result in results:
    statement_dict = {}
    statement = result.text
    

#     statement = re.findall("(.*)", statement, re.IGNORECASE)[0]
    statement_dict.update({"statement":statement})

    statement_list.append(statement_dict)

print(statement_list)

[{'statement': 'Date of Execution:'}, {'statement': ' April 25, 2018'}, {'statement': ' Offender:  '}, {'statement': 'Davila, Erick Daniel  '}, {'statement': 'Last Statement:  '}, {'statement': 'Yes, I would like to say nephew it burns huh.\xa0  You know I might have lost the fight but I’m still a soldier.\xa0 I still  love you all.\xa0 To my supporters and family y’all hold it down.\xa0 Ten  Toes down right. That’s all.'}]


In [155]:
def get_statement(row):
    html = requests.get('http://www.tdcj.state.tx.us/death_row/' + row['url']).content
    doc = BeautifulSoup(html, "html.parser")
    
    results = doc.find('div', attrs={'id':'content_right'}).find_all('p')  

    
    try:
        for result in results:
            statement = result.text
            statement_list.append(statement)
            statement_word = ''.join(statement_list)
        return pd.Series({
            'statement':statement_word
       })
    except:
        return pd.Series({})

In [156]:
df_all = df1.apply(get_statement, axis=1).join(df1)

In [157]:
df_all

,age,date,firstname,lastname,race,url
0,34,7/17/2018,Christopher,Young,Black,dr_info/youngchristopherlast.html
1,66,6/27/2018,Danny,Bible,White,dr_info/bibledannylast.html
2,37,5/16/2018,Juan,Castillo,Hispanic,dr_info/castillojuanlast.html
3,31,4/25/2018,Erick,Davila,Black,dr_info/davilaericklast.html
4,38,3/27/2018,Rosendo,"Rodriguez, III",Histpanic,dr_info/rodriguezrosendolast.html
5,62,2/1/2018,John,Battaglia,White,dr_info/battagliajohnlast.html
6,64,1/30/2018,William,Rayford,Black,dr_info/rayfordwilliamlast.html
7,55,1/18/2018,Anthony,Shore,White,dr_info/shoreanthonylast.html
8,47,11/08/2017,Ruben,Cardenas,Hispanic,dr_info/cardenasrubenlast.html
9,38,10/12/2017,Robert,Pruett,White,dr_info/pruettrobertlast.html


In [132]:
df_all['statement'][551]

'Last Statement:This offender declined to make a last statement.  '